In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [246]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Municipio','MEDU161A','MEDU161b'])
df18

,Cod Municipio,MEDU161A,MEDU161b
0,1100015,551,2001
1,1100023,1305,2007
2,1100031,616,2016
3,1100049,2736,2010
4,1100056,1947,2011
...,...,...,...
5565,5222005,892,2009
5566,5222054,574,2011
5567,5222203,222,2009
5568,5222302,-,NaN


In [247]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Educação', usecols=['CodMun','Medu161a','MEDU161b'])
df21

,CodMun,Medu161a,MEDU161b
0,1100015,-,-
1,1100023,954,2002
2,1100031,616,2009
3,1100049,2736,2010
4,1100056,1947,2011
...,...,...,...
5565,5222005,879,2008
5566,5222054,574,2011
5567,5222203,222,2009
5568,5222302,414,2009


In [248]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf 

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


Renomeando as colunas

In [249]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MEDU161A':'lei_criacao_plano_carreira',
                        'MEDU161b':'ano_criacao_lei'}) 

In [250]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                           'Medu161a':'lei_criacao_plano_carreira',
                           'MEDU161b':'ano_criacao_lei'}) 

In [251]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [252]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [253]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [254]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [255]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [256]:
df = df.drop(['CodMun'], axis=1) #eliminando coluna repetida

In [257]:
df= df.rename(columns={'UF':'sigla_uf',
                'Cod UF':'cod_uf',
                'Mun':'nome_municipio'}) #padronizando

In [258]:
#padronizando dados das colunas
df['lei_criacao_plano_carreira']=np.where(df['lei_criacao_plano_carreira']=='-','Sem dados',df['lei_criacao_plano_carreira'])
df['lei_criacao_plano_carreira']=np.where(df['lei_criacao_plano_carreira']=='(*) Não soube informar','Sem dados',df['lei_criacao_plano_carreira'])

df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='(*) Não soube informar','Sem dados',df['ano_criacao_lei'])
df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='Não informou','Sem dados',df['ano_criacao_lei'])
df['ano_criacao_lei']=np.where(df['ano_criacao_lei']=='-','Sem dados',df['ano_criacao_lei'])
df['ano_criacao_lei'] = df['ano_criacao_lei'].fillna('Sem dados') #usando metodo diferente para substituir nan, mas com o mesmo intuito de obter 'sem dados'

In [270]:
#transformando todos os valores das colunas abaixo em string
df['ano_criacao_lei'] = df['ano_criacao_lei'].astype(str)
df['lei_criacao_plano_carreira'] = df['lei_criacao_plano_carreira'].astype(str)


In [261]:
df = df[['ano','sigla_uf','id_municipio','nome_municipio','ano_criacao_lei','lei_criacao_plano_carreira']] #organizando as colunas


In [273]:
df.dtypes

ano                            int64
sigla_uf                      object
id_municipio                   int64
nome_municipio                object
ano_criacao_lei               object
lei_criacao_plano_carreira    object
dtype: object

Subindo para o GBQ

In [263]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

In [271]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Identificador do município pelo IBGE'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),       
        bigquery.SchemaField('ano_criacao_lei','STRING',description='Ano de criação da lei do plano de carreira para magistério'),
        bigquery.SchemaField('lei_criacao_plano_carreira','STRING',description='Lei de criação de plano de carreira para magistério')]

In [272]:
table_ref = dataset_ref.table('MUNIC_lei_plano_carreira_por_ano')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=1cb704e8-528f-4c59-a3e3-00a44993cb48>